## Weather split by year

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point
import pyproj
from fiona.crs import from_epsg
import numpy as np
import openpyxl

In [2]:
geo_shp = gpd.read_file("../geo_data/geo_final.shp", encoding="euc-kr")
geo_shp.head()

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,SG_NM,latitude,longitude,geometry
0,11110,Jongno-gu,종로구,서울특별시,37.594920,126.977317,"POLYGON ((127.00864 37.58047, 127.00871 37.580..."
1,11140,Jung-gu,중구,서울특별시,37.560143,126.995968,"POLYGON ((127.02314 37.57196, 127.02336 37.571..."
2,11170,Yongsan-gu,용산구,서울특별시,37.531386,126.979908,"POLYGON ((126.96918 37.55566, 126.96917 37.554..."
3,11200,Seongdong-gu,성동구,서울특별시,37.551030,127.041059,"POLYGON ((127.04341 37.57234, 127.04524 37.571..."
4,11215,Gwangjin-gu,광진구,서울특별시,37.546722,127.085746,"POLYGON ((127.10166 37.57240, 127.10224 37.572..."


In [3]:
geo_sig_id = list(geo_shp["SIG_CD"])
geo_sg_n = list(geo_shp["SG_NM"])
geo_sig_n = list(geo_shp["SIG_KOR_NM"])

geo_n_l = [[geo_sg_n[i], geo_sig_n[i]] for i in range(len(geo_sg_n))]

sig_df = dict(zip(geo_sig_id, geo_n_l))
keys = list(sig_df.keys())
print(len(keys), type(keys[0]))

250 <class 'str'>


In [6]:
def year_df(year, weather, target):
    year_df = weather[weather["year"]==year]
    year_df = year_df.groupby("SIG_CD").mean()
    year_df.drop(["year"], axis=1, inplace=True)
    sg_l = [sig_df[str(key)][0] for key in year_df.index]
    sig_l = [sig_df[str(key)][1] for key in year_df.index]
    year_df["SG_NM"] = sg_l
    year_df["SIG_KOR_NM"] = sig_l
    year_df["year"] = [year] * len(sg_l)
    year_df.reset_index(inplace=True)
    year_df.head()
    year_df.to_csv(f"./{year}_{target}.csv")

In [7]:
def make_year_df(path,year_l):
    target = path.split("/")[-1].split(".")[0]
    weather = pd.read_csv(path)
    weather.drop(["SD_ENG_NM"], axis=1, inplace=True)
    weather["year"] = list(int(i) for i in weather["year"])
    for year in year_l:
        year_df(year, weather, target)

In [8]:
year_l = [i for i in range(2014,2019)]

In [9]:
path = "./pre/precipitation.csv"
make_year_df(path,year_l)

In [10]:
path = "./pre/pressure.csv"
make_year_df(path,year_l)

In [11]:
path = "./pre/temperature.csv"
make_year_df(path,year_l)